In [1]:
# imports
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up the executable path needed to scrape the webpage
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_mac64.zip
Driver has been saved in cache [/Users/matthewgodfrey/.wdm/drivers/chromedriver/mac64/95.0.4638.69]


In [3]:
# Website to scrape
url = "https://redplanetscience.com/"
browser.visit(url)

In [7]:
# Find the first article title and summary paragraph

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the first element with the news title, and the first element with the paragraph text
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

NASA Readies Perseverance Mars Rover's Earthly Twin 
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


In [8]:
# Quit the browser
browser.quit()